In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import gc
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
import time
from datetime import timedelta, date
from fastai.tabular import *

%matplotlib inline

In [2]:
train_df = pd.read_csv('train.csv')
test_df =  pd.read_csv('test.csv')

In [3]:
train_df.columns

Index(['application_date', 'segment', 'branch_id', 'state', 'zone',
       'case_count'],
      dtype='object')

In [4]:
print(train_df.shape)
print(test_df.shape)

(80402, 6)
(180, 3)


In [5]:
train_df.tail()

,application_date,segment,branch_id,state,zone,case_count
80397,2019-07-19,2,NaN,WEST BENGAL,NaN,1886.0
80398,2019-07-20,2,NaN,WEST BENGAL,NaN,1480.0
80399,2019-07-21,2,NaN,WEST BENGAL,NaN,1028.0
80400,2019-07-22,2,NaN,WEST BENGAL,NaN,1946.0
80401,2019-07-23,2,NaN,WEST BENGAL,NaN,1984.0


In [6]:
train_s1 = train_df[train_df['segment']==1]
train_s2 = train_df[train_df['segment']==2]

In [7]:
train_s1 = pd.DataFrame(train_s1.groupby(['application_date'])['case_count'].sum())
train_s1.reset_index(inplace=True)
train_s2 = pd.DataFrame(train_s2.groupby(['application_date'])['case_count'].sum())
train_s2.reset_index(inplace=True)

In [8]:
train_s1.head()

,application_date,case_count
0,2017-04-01,299.0
1,2017-04-03,42.0
2,2017-04-04,23.0
3,2017-04-05,1530.0
4,2017-04-07,1341.0


In [9]:
test_df1 = test_df[test_df['segment']==1].drop(['id','segment'],axis=1)
test_df2 = test_df[test_df['segment']==2].drop(['id','segment'],axis=1)

In [10]:
test_df1.head()

,application_date
0,2019-07-06
1,2019-07-07
2,2019-07-08
3,2019-07-09
4,2019-07-10


In [11]:
add_datepart(train_s1, 'application_date', drop = False)
add_datepart(train_s2, 'application_date', drop = False)

add_cyclic_datepart(train_s1, 'application_date')
add_cyclic_datepart(train_s2, 'application_date')

add_datepart(test_df1, 'application_date', drop = False)
add_datepart(test_df2, 'application_date', drop = False)

add_cyclic_datepart(test_df1, 'application_date')
add_cyclic_datepart(test_df2, 'application_date')

,application_Year,application_Month,application_Week,application_Day,application_Dayofweek,application_Dayofyear,application_Is_month_end,application_Is_month_start,application_Is_quarter_end,application_Is_quarter_start,...,application_Is_year_start,application_Elapsed,application_weekday_cos,application_weekday_sin,application_day_month_cos,application_day_month_sin,application_month_year_cos,application_month_year_sin,application_day_year_cos,application_day_year_sin
87,2019,7,30,24,2,205,False,False,False,False,...,False,1563926400,-0.222521,0.974928,-0.050649,-0.998717,-1.000000e+00,1.224647e-16,-0.932289,-0.361714
88,2019,7,30,25,3,206,False,False,False,False,...,False,1564012800,-0.900969,0.433884,0.151428,-0.988468,-1.000000e+00,1.224647e-16,-0.925925,-0.377708
89,2019,7,30,26,4,207,False,False,False,False,...,False,1564099200,-0.900969,-0.433884,0.347305,-0.937752,-1.000000e+00,1.224647e-16,-0.919286,-0.393590
90,2019,7,30,27,5,208,False,False,False,False,...,False,1564185600,-0.222521,-0.974928,0.528964,-0.848644,-1.000000e+00,1.224647e-16,-0.912375,-0.409356
91,2019,7,30,28,6,209,False,False,False,False,...,False,1564272000,0.623490,-0.781831,0.688967,-0.724793,-1.000000e+00,1.224647e-16,-0.905193,-0.425000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2019,10,42,20,6,293,False,False,False,False,...,False,1571529600,0.623490,-0.781831,-0.758758,-0.651372,-1.836970e-16,-1.000000e+00,0.309017,-0.951057
176,2019,10,43,21,0,294,False,False,False,False,...,False,1571616000,1.000000,0.000000,-0.612106,-0.790776,-1.836970e-16,-1.000000e+00,0.325342,-0.945596
177,2019,10,43,22,1,295,False,False,False,False,...,False,1571702400,0.623490,0.781831,-0.440394,-0.897805,-1.836970e-16,-1.000000e+00,0.341571,-0.939856
178,2019,10,43,23,2,296,False,False,False,False,...,False,1571788800,-0.222521,0.974928,-0.250653,-0.968077,-1.836970e-16,-1.000000e+00,0.357698,-0.933837


In [12]:
train_s1.columns

Index(['case_count', 'application_Year', 'application_Month',
       'application_Week', 'application_Day', 'application_Dayofweek',
       'application_Dayofyear', 'application_Is_month_end',
       'application_Is_month_start', 'application_Is_quarter_end',
       'application_Is_quarter_start', 'application_Is_year_end',
       'application_Is_year_start', 'application_Elapsed',
       'application_weekday_cos', 'application_weekday_sin',
       'application_day_month_cos', 'application_day_month_sin',
       'application_month_year_cos', 'application_month_year_sin',
       'application_day_year_cos', 'application_day_year_sin'],
      dtype='object')

In [13]:
train_s1_x = train_s1.drop(['case_count'],axis = 1)
train_s2_x = train_s2.drop(['case_count'],axis = 1)

train_s1_y = train_s1['case_count']
train_s2_y = train_s2['case_count']

In [14]:
cont_names, cat_names = cont_cat_split(df=train_s1_x, max_card=100)

In [15]:
tfm = FillMissing(cat_names, cont_names)
tfm(train_s1_x)
tfm(train_s2_x)

In [16]:
# norm = Normalize(cat_list, cont_list)
# norm.apply_train(train_s1_x)
# norm.apply_train(train_s2_x)
# norm.apply_test(test_df1)
# norm.apply_test(test_df2)

In [17]:
train_s1_x_early, valid_s1_x_early, train_s1_y_early, valid_s1_y_early = train_test_split(train_s1_x,train_s1_y,test_size = 0.2)
train_s2_x_early, valid_s2_x_early, train_s2_y_early, valid_s2_y_early = train_test_split(train_s2_x,train_s2_y,test_size = 0.2)

In [18]:
train_s2_x.columns

Index(['application_Year', 'application_Month', 'application_Week',
       'application_Day', 'application_Dayofweek', 'application_Dayofyear',
       'application_Is_month_end', 'application_Is_month_start',
       'application_Is_quarter_end', 'application_Is_quarter_start',
       'application_Is_year_end', 'application_Is_year_start',
       'application_Elapsed', 'application_weekday_cos',
       'application_weekday_sin', 'application_day_month_cos',
       'application_day_month_sin', 'application_month_year_cos',
       'application_month_year_sin', 'application_day_year_cos',
       'application_day_year_sin'],
      dtype='object')

In [19]:
# procs = [FillMissing, Categorify, Normalize]
# path=''
# dep_var = 'case_count'
# start_indx = len(train_s1) - int(len(train_s1) * 0.2)
# end_indx = len(train_s1)

# val1 = (TabularList.from_df(train_s1.iloc[start_indx:end_indx].copy(), path=path, cat_names=cat_names, cont_names=cont_names))

# #test1 = (TabularList.from_df(test_df1, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs))

# data1 = (TabularList.from_df(train_s1, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
#                              .split_by_idx(list(range(start_indx,end_indx)))                  
#                              .label_from_df(cols=dep_var)
#                              #.add_test(test1)
#                              .databunch())

In [20]:
# learn = tabular_learner(data1, layers=[300,200, 100, 50],metrics= [rmse,r2_score])
# learn.lr_find(start_lr = 1e-05,end_lr = 1e+05, num_it = 100)
# learn.recorder.plot()

In [21]:
# learn.fit_one_cycle(25)

In [22]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [23]:
from sklearn.ensemble import RandomForestRegressor
reg1 = RandomForestRegressor(n_estimators = 100,max_depth =6)
reg1.fit(train_s1_x_early, train_s1_y_early)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [24]:
train_s1_pred_early = reg1.predict(train_s1_x_early)
train_s1_pred_valid = reg1.predict(valid_s1_x_early)


In [25]:
print('Train mape: {}'.format(mean_absolute_percentage_error(train_s1_y_early,train_s1_pred_early)))
print('Valid mape: {}'.format(mean_absolute_percentage_error(valid_s1_y_early,train_s1_pred_valid)))

Train mape: 271.09702950526037
Valid mape: 517.6099000183714


In [26]:
from sklearn.ensemble import RandomForestRegressor
reg2 = RandomForestRegressor(n_estimators = 100,max_depth =6)
reg2.fit(train_s2_x_early, train_s2_y_early)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [27]:
train_s2_pred_early = reg2.predict(train_s2_x_early)
train_s2_pred_valid = reg2.predict(valid_s2_x_early)

In [28]:
print('Train mape: {}'.format(mean_absolute_percentage_error(train_s2_y_early,train_s2_pred_early)))
print('Valid mape: {}'.format(mean_absolute_percentage_error(valid_s2_y_early,train_s2_pred_valid)))

Train mape: 20.261074347084772
Valid mape: 25.67615183135158
